In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import json
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db+'?sslmode=disable', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [2]:
db, conn = pgconnect(credentials)

Connected successfully.


In [3]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [4]:
def split(word):
    return [char for char in word]
def toInt(st):
    parts=split(st)
    if parts.count(",")>0:
        parts.remove(",")
    i=0
    num=0
    while i<len(parts):
        num+=10**(len(parts)-i-1)* int(parts[i])
        i+=1
    return num

In [5]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [6]:
conn.execute("""
DROP TABLE IF EXISTS BusinessStats;
CREATE TABLE BusinessStats(
   area_id INTEGER,
   area_name VARCHAR(50),
   number_of_businesses INTEGER,
   accommodation_and_food_services INTEGER,
   retail_trade INTEGER,
   agriculture_forestry_and_fishing INTEGER,
   health_care_and_social_assistance INTEGER,
   public_administration_and_safety INTEGER,
   transport_postal_and_warehousing INTEGER
);""")


In [7]:
Businessdata = pd.read_csv('BusinessStats.csv')
Businessdata.to_sql("businessstats", con=conn, if_exists='append', index=False)

301

In [8]:
conn.execute("""
DROP TABLE IF EXISTS Neighbourhoods;
CREATE TABLE Neighbourhoods(
   area_id INTEGER,
   area_name VARCHAR(50),
   land_area FLOAT,
   population VARCHAR(50),
   number_of_dwellings VARCHAR(50),
   number_of_businesses INTEGER,
   median_annual_household_income INTEGER,
   avg_monthly_rent INTEGER,
   "0_4" INTEGER,
   "5_9" INTEGER,
   "10_14" INTEGER,
   "15_19" INTEGER
);""")
Neighbourhoodsdata = pd.read_csv('Neighbourhoods.csv')
Neighbourhoodsdata.fillna(0, inplace = True)
Neighbourhoodsdata.to_sql("neighbourhoods", con=conn, if_exists='append', index=False)

322

In [9]:
SA2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")
SA2_data = SA2_data.replace(to_replace='None', value=np.nan).dropna()
SA2_data

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58423 -35.44427, 149.58444 -35.4..."
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21898 -35.36739, 149.21799 -35.3..."
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,"POLYGON ((149.21325 -35.34325, 149.21619 -35.3..."
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,"POLYGON ((149.24033 -35.34782, 149.24023 -35.3..."
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,"POLYGON ((149.23580 -35.38738, 149.23771 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,"POLYGON ((148.80406 -35.37620, 148.80416 -35.3..."
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67392 -10.41567, 105.67398 -10.4..."
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,"MULTIPOLYGON (((96.83047 -12.17636, 96.83045 -..."
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,"MULTIPOLYGON (((150.69566 -35.18297, 150.69555..."


In [10]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [11]:
SA2_og = SA2_data.copy()  # creating a copy of the original for later
SA2_data['geom'] = SA2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4283))  # applying the function
SA2_data = SA2_data.drop(columns="geometry")  # deleting the old copy
SA2_data

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,MULTIPOLYGON (((149.5842329970001 -35.44426999...
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,MULTIPOLYGON (((149.2189819130001 -35.36739402...
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,MULTIPOLYGON (((149.2132479820001 -35.34324799...
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,MULTIPOLYGON (((149.240332114 -35.347822616999...
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,MULTIPOLYGON (((149.235800967 -35.387375302999...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,MULTIPOLYGON (((148.804064002 -35.376204003999...
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,MULTIPOLYGON (((105.6739190100001 -10.41567300...
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,MULTIPOLYGON (((96.83046599400006 -12.17635995...
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,MULTIPOLYGON (((150.6956629970001 -35.18296700...


In [12]:
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    "SA2_MAIN16" NUMERIC, 
    "SA2_5DIG16" NUMERIC, 
    "SA2_NAME16" VARCHAR(50), 
    "SA3_CODE16" NUMERIC, 
    "SA3_NAME16" VARCHAR(50),
    "SA4_CODE16" NUMERIC,
    "SA4_NAME16" VARCHAR(50),
    "GCC_CODE16" VARCHAR(10),
    "GCC_NAME16" VARCHAR(50),
    "STE_CODE16" NUMERIC,
    "STE_NAME16" VARCHAR(50),
    "AREASQKM16" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);""")

In [ ]:
SA2_data.to_sql("sa2", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from sa2")

In [ ]:
schoolsdata = pd.concat(
map(gpd.read_file, ['school_catchments/catchments_primary.shp', 'school_catchments/catchments_secondary.shp', 'school_catchments/catchments_future.shp']), ignore_index=True)
schoolsdata

In [ ]:
schoolsdata_og = schoolsdata.copy()  
schoolsdata['geom'] = schoolsdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4283))  
schoolsdata = schoolsdata.drop(columns="geometry") 
schoolsdata

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS schools;
CREATE TABLE schools (
    "USE_ID" NUMERIC, 
    "CATCH_TYPE" VARCHAR(20), 
    "USE_DESC" VARCHAR(50), 
    "ADD_DATE" NUMERIC, 
    "KINDERGART" VARCHAR(10),
    "YEAR1" VARCHAR(10),
    "YEAR2" VARCHAR(10),
    "YEAR3" VARCHAR(10),
    "YEAR4" VARCHAR(10),
    "YEAR5" VARCHAR(10),
    "YEAR6" VARCHAR(10),
    "YEAR7" VARCHAR(10),
    "YEAR8" VARCHAR(10),
    "YEAR9" VARCHAR(10),
    "YEAR10" VARCHAR(10),
    "YEAR11" VARCHAR(10),
    "YEAR12" VARCHAR(10),
    "PRIORITY" VARCHAR(10),
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [ ]:
schoolsdata.to_sql("schools", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from schools")

In [ ]:
breakdata = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")
breakdata = breakdata.replace(to_replace='None', value=np.nan).dropna()
breakdata

In [ ]:
srid=4326
breakdata_og = breakdata.copy()  
breakdata['geom'] = breakdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  
breakdata = breakdata.drop(columns="geometry") 
breakdata

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS break_enter;
CREATE TABLE break_enter (
    "OBJECTID" NUMERIC, 
    "Contour" NUMERIC, 
    "Density" VARCHAR(20), 
    "ORIG_FID" NUMERIC, 
    "Shape_Leng" NUMERIC,
    "Shape_Area" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4326)
);"""
)

In [ ]:
breakdata.to_sql("break_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from break_enter")

In [ ]:
bicycle_data = gpd.read_file('Bicycle_count_sites.geojson')
bicycle_data

In [ ]:
bicycle_data['geom'] = bicycle_data['geometry'].apply(lambda x: WKTElement(x.wkt, 4283))
bicycle_data = bicycle_data.drop(columns="geometry")
bicycle_data

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS bicycles;
CREATE TABLE bicycles (
    "OBJECTID" NUMERIC, 
    "SiteID" NUMERIC, 
    "Intersection" VARCHAR(150),
    geom GEOMETRY(POINT,4283)
);"""
)

In [ ]:
bicycle_data.to_sql('bicycles', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from bicycles")


In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
playground_data = gpd.read_file('Playgrounds.kml', driver='KML')
playground_data

In [ ]:
playground_data['geom'] = playground_data['geometry'].apply(lambda x: WKTElement(x.wkt, 4283))
playground_data = playground_data.drop(columns="geometry")
playground_data

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS playgrounds;
CREATE TABLE playgrounds (
    "Name" VARCHAR(80), 
    "Description" VARCHAR(50), 
    geom GEOMETRY(POINT,4283)
);"""
)

In [ ]:
playground_data.to_sql('playgrounds', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from playgrounds")